In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!pip freeze

In [ ]:
!pip install PyPDF2

import PyPDF2
import re
import pandas as pd

In [ ]:
from google.colab import files
import PyPDF2

# Upload the PDF file
uploaded = files.upload()

# Get the filename from the uploaded file
filename = next(iter(uploaded))

def convert_pdf_to_text_py(pdf_path):
    """Converts a PDF file to plain text while preserving the original formatting.

    Args:
        pdf_path: The path to the PDF file.

    Returns:
        The extracted text from the PDF file with formatting preserved as much as possible.
    """

    content = ""
    with open(pdf_path, "rb") as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)

        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            page_content = page.extract_text()

            # Split the page content into lines
            lines = page_content.splitlines()

            # Add each line to the content with a newline
            for line in lines:
                content += line + "\n"

            # Add an extra newline to separate pages
            content += "\n"

    return content

# Convert the uploaded PDF to text
extracted_text = convert_pdf_to_text_py(filename)
print(extracted_text)

Saving CED.pdf to CED (1).pdf
Senior Front -end Angular Developer  
• CED  
• Tunis, Tunisie  
 
Postes vacants:  
1 poste ouvert  
Type d'emploi désiré :  
CDI 
Experience :  
3 à 5 ans  
Niveau d'étude :  
Ingénieur  
Langue :  
Français, Anglais  
Genre :  
Indifférent  
Description de l'emploi  
CED Group is an European service provider for many insurers, property managers, corporates and 
governments. Its business consists on protecting and restoring value for the citizens all over 14 
European countries. 2000+ employees and experts are serving our  customers all over Europe. 
CED Tunisia is a subsidiary of CED Group. It consists of an IT -BI Development center and a 
Businees Process Operations (BPO) team to support our various European offices and drive 
operational excellence. We employ 110 profession als as of now and going to recruit 20 new 
positions in the coming 3 months, 5 of them in IT. We are building in Tunis the digitalisation of all 
our front office applications and

In [ ]:
!python -m spacy download fr_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 17.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
import re
import pickle


nlp = spacy.load("fr_core_news_md")

# Load trained models
with open("job_title_FFFfinalmodel.pkl", "rb") as file:
    job_title_model = pickle.load(file)

with open("company_name_FFFfinalmodel.pkl", "rb") as file:
    company_name_model = pickle.load(file)


def extract_job_info(text):
    info = {
        'job_title': None,
        'company_name': None,
        'location': None,
        'salary': None,
        'benefits': None,
        'duration': None,
        'skills_qualifications': None,
        'opportunities': None,
        'contract_type': None,
        'language': [],
        'genre': None,
        'experience': None,
        'date_of_beginning': None,
        'date_of_expiration': None,# Added for expiration date
        'phone_number': None,  # Added for phone number
        'email': None  # Added for email address
    }

    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]
    lines = text.strip().split('\n')

     # Predict Job Title using the trained model
    info['job_title'] = job_title_model.predict([text])[0]

    # Predict Company Name using the trained model
    info['company_name'] = company_name_model.predict([text])[0]


    # Extract Location
    for ent in doc.ents:
        if ent.label_ == "LOC":
            info['location'] = ent.text
            break  # Stop after finding the first LOC entity

    # Extract Date of Beginning (No change)
    date_pattern = re.compile(r'\b\d{1,2}\s+(?:janvier|février|mars|avril|mai|juin|juillet|août|septembre|'
                             r'octobre|novembre|décembre)\b', re.IGNORECASE)
    date_match = date_pattern.search(text)
    if date_match:
        info['date_of_beginning'] = date_match.group(0)

    # Extract Date of Expiration (Added)
    expiration_pattern = re.compile(r'Date d\'expiration\s*:\s*(\d{1,2}/\d{1,2}/\d{4})', re.IGNORECASE)
    expiration_match = expiration_pattern.search(text)
    if expiration_match:
        info['date_of_expiration'] = expiration_match.group(1)

    # Extract Language (No change)
    language_keywords = ['français', 'anglais','allemand','arabe']
    for keyword in language_keywords:
        if keyword.lower() in text.lower():
            info['language'].append (keyword)


    # Extract Genre (No change)
    genre_keywords = ['homme', 'femme', 'indifférent']
    for keyword in genre_keywords:
        if keyword.lower() in text.lower():
            info['genre'] = keyword
            break

    # Extract Experience (No change)
    experience_pattern = re.compile(r'\b\d+\s+(?:ans?)\b', re.IGNORECASE)
    experience_match = experience_pattern.search(text)
    if experience_match:
        info['experience'] = experience_match.group(0)

    # Extract Salary (Improved)
    salary_pattern = re.compile(r'\d+\s?(?:K|k|€|dollars?|euros?)\b', re.IGNORECASE)  # More general pattern
    salary_match = salary_pattern.search(text)
    if salary_match:
        info['salary'] = salary_match.group(0)

    # Extract Duration (No change)
    duration_pattern = re.compile(r'\b\d+\s+(?:ans?|mois?|semaines?)\b', re.IGNORECASE)
    duration_match = duration_pattern.search(text)
    if duration_match:
        info['duration'] = duration_match.group(0)

    # Extract Contract Type (No change)
    contract_pattern = re.compile(r'\b(?:CDI|CDD|SIVP|stage|freelance)\b', re.IGNORECASE)
    contract_match = contract_pattern.search(text)
    if contract_match:
        info['contract_type'] = contract_match.group(0)

    # Extract Skills and Qualifications (New)
    # Liste prédéfinie de compétences techniques
    skills_list = [
    'Python', 'Java',  'C++','C#', 'Javascript', 'HTML', 'CSS','SASS','Bootstrap', 'SQL', 'MATLAB', 'Symfony','AJAX','Jquery','Laravel',
    'Ruby', 'PHP', 'Swift', 'Kotlin', 'Scala', 'Perl', 'TypeScript', 'Django','.Net'
    'Flask', 'React', 'Angular', 'Vue.js', 'TensorFlow', 'PyTorch', 'Docker', 'Kubernetes',
    'Git', 'Linux', 'AWS', 'Azure', 'GCP', 'Hadoop', 'Spark','SQL','telecoms','Communication','problem solving','learning skills','design patterns'
     ]

    skills_qualifications = []

    # Identifier les compétences présentes dans le texte
    for skill in skills_list:
        if skill.lower() in text.lower():  # Recherche insensible à la casse
            skills_qualifications.append(skill)

# Supprimer les doublons (au cas où) et stocker les compétences extraites
    info['skills_qualifications'] = list(set(skills_qualifications))


    # Extract Benefits (No change)
    benefits_keywords = ['avantages', 'benefices']
    benefits = []
    for sent in doc.sents:
        if any(keyword in sent.text.lower() for keyword in benefits_keywords):
            benefits.append(sent.text)
    info['benefits'] = ' '.join(benefits).strip()

    # Extract Opportunities (No change)
    opportunities_keywords = ['opportunité', 'croissance', 'carrière']
    opportunities = []
    for sent in doc.sents:
        if any(keyword in sent.text.lower() for keyword in opportunities_keywords):
            opportunities.append(sent.text)
    info['opportunities'] = ' '.join(opportunities).strip()

    # Extract Phone Number (New)
    phone_pattern = re.compile(r'\b(\+?[\d\s-]{7,15})\b', re.IGNORECASE)
    phone_match = phone_pattern.search(text)
    if phone_match:
        info['phone_number'] = phone_match.group(0)

    # Extract Email Address (New)
    email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b', re.IGNORECASE)
    email_match = email_pattern.search(text)
    if email_match:
        info['email'] = email_match.group(0)

    return info

In [ ]:
# Example usage
extracted_info = extract_job_info(extracted_text)
print(extracted_info)

{'job_title': 'Senior Front-end Angular Developer', 'company_name': 'CED', 'location': 'Tunis', 'salary': None, 'benefits': '', 'duration': '5 ans', 'skills_qualifications': ['Git', 'HTML', 'problem solving', 'Java', 'SQL', 'C#', 'design patterns', 'Azure', 'TypeScript', 'Angular', 'Bootstrap', 'learning skills', 'CSS', 'SASS', 'Javascript'], 'opportunities': '', 'contract_type': 'CDI', 'language': ['français', 'anglais'], 'genre': 'indifférent', 'experience': '5 ans', 'date_of_beginning': None, 'date_of_expiration': None, 'phone_number': None, 'email': 'MOUNA.GHRIBI@CED.GROUP'}


In [ ]:
#Converting extracted informations to a DataFrame

import pandas as pd

data = {
      "job_title": extracted_info['job_title'],
      "company_name": extracted_info['company_name'],
      "location": extracted_info['location'],
      "salary": extracted_info['salary'],
      "benefits": extracted_info['benefits'],
      "duration": extracted_info['duration'],
      "skills_qualifications": extracted_info['skills_qualifications'],
      "opportunities": extracted_info['opportunities'],
      "contract_type": extracted_info['contract_type'],
      "language": extracted_info['language'],
      "genre": extracted_info['genre'],
      "experience": extracted_info['experience'],
      "date_of_beginning": extracted_info['date_of_beginning'],
      "date_of_expiration": extracted_info['date_of_expiration']
    }

df = pd.DataFrame([data])
#df.to_csv('extracted_info.csv', index=False)
df


,job_title,company_name,location,salary,benefits,duration,skills_qualifications,opportunities,contract_type,language,genre,experience,date_of_beginning,date_of_expiration
0,Data Engineer,IT Services,Tunis,None,- Des avantages sociaux : des tickets restaura...,5 ans,Exigences de l'emploi \nCompétences requises ...,,CDI,"[français, anglais, arabe]",indifférent,5 ans,None,None
